In [ ]:
from tempfile import TemporaryDirectory
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

from calibrations.matsch_b2 import obs, model

In [ ]:
targets = [
    ('rainfall_amount', 'mm', 'Rainfall Amount'),
    ('wind_speed', 'm/s', 'Wind Speed'),
    ('relative_humidity', None, 'Relative Humidity'),
     ('air_temperature', 'C', 'Air Temperature'),
    ('surface_downwelling_shortwave_flux', 'W/m2', 'Downwelling Shortwave Flux'),
    ('soil_moisture_content_50', None, 'Soil Moisture at 50mm'),
    ('soil_moisture_content_200', None, 'Soil Moisture at 200mm'),
    ('latent_heat_flux_in_air', 'W/m2', 'Latent Heat Flux'),
    ('sensible_heat_flux_in_air', 'W/m2', 'Sensible Heat Flux')
]

In [ ]:
with TemporaryDirectory() as tmpdir:
    sim = model.eval(tmpdir)

In [ ]:
for name, unit, desc in targets:
    display(Markdown(f"# {desc}"))
    print(f"Loss for {name} is {obs.metric(name, sim)}")
    obs.compare(name, sim, desc=desc, unit=unit)
    plt.show()